In [ ]:
!export CUDA_VISIBLE_DEVICES=3

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
import pdb

In [ ]:
hyp = {
    'dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
#     'base_folder_dm_info_list': [
#         {
#             'path': '/mnt/data/wsj0.8k/si_tr_s/',
#             'ext': 'wav',
#             'type': 'clean',
#         }
#     ],
#     'data_clean_prob': 1.,
    'data_folder': '/mnt/data/Speech/wsj_tse',
    'valid_data': '/mnt/data/Speech/wsj_tse/mix_2_spk_tt/meta.csv',
    'sample_rate': 8000,
    'training_signal_len': 24000,
}

In [ ]:
from data_mixing import static_mixing_prep
import torch
import numpy as np
from speechbrain.dataio.batch import PaddedBatch

# Old version

In [ ]:
ds = static_mixing_prep(hyp, 'valid')
dl = torch.utils.data.DataLoader(
        ds,
        batch_size=hyp["dataloader_opts"]["batch_size"],
        num_workers=hyp["dataloader_opts"]["num_workers"],
        collate_fn=PaddedBatch,
        worker_init_fn=lambda x: np.random.seed(
            int.from_bytes(os.urandom(4), "little") + x
        ),
    )

In [ ]:
def listen_batch(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    diff = mix - s1 - s2
    print(abs(mix).max())
    print((diff**2).mean())
    sound(mix[0].numpy(), rate=hyp['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hyp['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hyp['sample_rate'], label='s2')
    sound(enr[0].numpy(), rate=hyp['sample_rate'], label='enr')

In [ ]:
for i, batch in enumerate(dl):  
    if i == 3:
        listen_batch(batch)
        break

# New version 

In [ ]:
hparams = {
    'dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'data_folder': '/mnt/data/wsj0.8k/',
    'valid_txtpath': '/mnt/data/Speech/wsj_tse/mix_2_spk_tt_extr.txt',
    'valid_data': './wsj_cv.csv',
    'sample_rate': 8000,
    'training_signal_len': 24000,
}

In [ ]:
from prepare_data import create_wsj_tse_csv
from data_mixing import static_data_prep


In [ ]:
create_wsj_tse_csv(hparams['data_folder'], hparams['valid_txtpath'], hparams['valid_data'])

In [ ]:
new_ds = static_data_prep(hparams, 'valid')
new_dl = torch.utils.data.DataLoader(
        new_ds,
        batch_size=hparams["dataloader_opts"]["batch_size"],
        num_workers=hparams["dataloader_opts"]["num_workers"],
        collate_fn=PaddedBatch,
        worker_init_fn=lambda x: np.random.seed(
            int.from_bytes(os.urandom(4), "little") + x
        ),
    )

In [ ]:
for i, new_batch in enumerate(new_dl):  
    if i == 0:
        listen_batch(new_batch)
        break